In [1]:
import os
import numpy as np

from lsst.sims.photUtils.Sed import Sed
from lsst.sims.photUtils.Bandpass import Bandpass

ModuleNotFoundError: No module named 'lsst.utils'

In [2]:
import os
import numpy as np

from lsst.sims.photUtils.Sed import Sed
from lsst.sims.photUtils.Bandpass import Bandpass

In [8]:
# See? Weird, right?

In [ ]:
# and if you try copypasting in the code from get_star_properties.py:

In [7]:
from lsst.utils import getPackageDir
from lsst.sims.catUtils.baseCatalogModels import StarObj
from lsst.sims.utils import ObservationMetaData
from lsst.sims.utils import equatorialFromGalactic

ModuleNotFoundError: No module named 'lsst.utils'

In [11]:
# And that one doesn't go anywhere if you try running the cell again.

In [ ]:
# And if you try getting around that by skipping that line, I get this error instead. I've looked, and
# lsst.sims.catUtils really doesn't seem to be on my computer anywhere.

In [10]:
from lsst.sims.catUtils.baseCatalogModels import StarObj
from lsst.sims.utils import ObservationMetaData
from lsst.sims.utils import equatorialFromGalactic

ModuleNotFoundError: No module named 'lsst.sims.catUtils'

## In conlusion, lots of fun at the moment!